## 2024년 1월 기준의 행정동 경계 x
- 부천시의 경우 24.1을 기준으로 행정동이 10 -> 37개로 늘어남
- 36개동이 존재하던 2019 2q의 데이터 중 부천시의 데이터만 필터링
- 2024.2의 행정경계(선거구정보 존재) 중 부천시 데이터 없애기
- 두 레이어 합치기
- 부천시 외 일부 행정동 합동, 명칭변경 반영 등 
- 부천시 및 위의 조정레이어의 선거구 정보는 QGIS를 통해 수기입력

In [20]:
#->선거구shp_22대총선
#qgis sgis의 2019_2q 파일 중 부천시 부분만 필터 후 27개 동으로 정비
#부천시 10개동 -> 37개동 반영
#분동/합동으로 인한 차이는 미반영 -> 부천시_반영후 
#->qgis를 이용한 가공-> 22대총선_선거구_alias 반영전 
import pandas as pd
import os
import shutil
pd.set_option('display.max_rows', 10000)
import numpy as np
import geopandas as gpd

In [23]:
#선거구정보가 모두 추가된 동 정보 가져오기 : 3528 - 5(강원도)
file = gpd.read_file('../shp/행정경계_202402/22대총선_선거구/22대총선_선거구_동/부천시_반영전')
gdf = gpd.GeoDataFrame(file, crs = file.crs)
gdf = gdf.set_crs(epsg=5186, allow_override=True)
print(len(gdf))
gdf.head()

3523


,ADM_NM,ADM_CD,SIGUNGU_NM,SIDO_NM,SIGUNGU_CD,SIDO_CD,SIDO_SIGUN,cor,DISTRICT,DIS,geometry
0,사직동,11010530,종로구,서울특별시,11010,11,서울특별시_종로구_사직동,0,종로구,서울특별시_종로구,"POLYGON ((197702.069 553187.311, 197703.431 55..."
1,삼청동,11010540,종로구,서울특별시,11010,11,서울특별시_종로구_삼청동,0,종로구,서울특별시_종로구,"POLYGON ((198170.457 553770.678, 198172.189 55..."
2,부암동,11010550,종로구,서울특별시,11010,11,서울특별시_종로구_부암동,0,종로구,서울특별시_종로구,"POLYGON ((196621.023 556395.880, 196628.323 55..."
3,평창동,11010560,종로구,서울특별시,11010,11,서울특별시_종로구_평창동,0,종로구,서울특별시_종로구,"POLYGON ((197800.719 559064.245, 197782.581 55..."
4,한남동,11030740,용산구,서울특별시,11030,11,서울특별시_용산구_한남동,0,용산구,서울특별시_용산구,"POLYGON ((200404.268 550031.323, 200404.019 55..."


In [22]:
#부천시 동정보 가져오기
file = gpd.read_file('../shp/행정경계_201905/bucheon_refined')
gdf_1 = gpd.GeoDataFrame(file, crs = file.crs)
gdf_1 = gdf_1.set_crs(epsg=5186)
print(len(gdf_1))
gdf_1.head()

37


,base_date,adm_dr_cd,adm_dr_nm,geometry
0,20190630,3105051,심곡2동,"POLYGON ((181312.979 543148.000, 181326.288 54..."
1,20190630,3105052,심곡1동,"POLYGON ((180885.579 544457.310, 180889.348 54..."
2,20190630,3105053,심곡3동,"POLYGON ((180386.450 543648.103, 180383.428 54..."
3,20190630,3105054,원미2동,"POLYGON ((181688.947 543267.074, 181670.647 54..."
4,20190630,3105055,소사동,"POLYGON ((182102.767 543316.488, 182087.079 54..."


In [24]:
#선거구_동 기준으로 컬럼 맞춰주기
li_col = list(gdf.columns)
li_col


['ADM_NM',
 'ADM_CD',
 'SIGUNGU_NM',
 'SIDO_NM',
 'SIGUNGU_CD',
 'SIDO_CD',
 'SIDO_SIGUN',
 'cor',
 'DISTRICT',
 'DIS',
 'geometry']

In [25]:
#컬럼맞춰주기
gdf_1 = gdf_1.rename({'adm_dr_cd':'ADM_CD','adm_dr_nm':'ADM_NM'}, axis = 1)
gdf_1['SIDO_NM'] = '경기도'
gdf_1['SIDO_CD'] = '31'
gdf_1['SIGUNGU_CD'] = '31050'
gdf_1['cor'] = '부천시'
gdf_1['SIGUNGU_NM'] = '0'
gdf_1['SIDO_SIGUN'] = '0'
gdf_1['DISTRICT'] = '0'
gdf_1['DIS'] = '0'
gdf_1 = gdf_1.drop('base_date', axis = 1)
gdf_1 = gdf_1[li_col]
gdf_1.columns

Index(['ADM_NM', 'ADM_CD', 'SIGUNGU_NM', 'SIDO_NM', 'SIGUNGU_CD', 'SIDO_CD',
       'SIDO_SIGUN', 'cor', 'DISTRICT', 'DIS', 'geometry'],
      dtype='object')

In [26]:
#부천시 제거
print(len(gdf))
gdf_2 = gdf.loc[~gdf['DISTRICT'].str.contains('부천시')]
print(len(gdf_2))

3523
3513


In [27]:
#좌표계 맞춰주기
gdf_3 = pd.concat([gdf_2, gdf_1], axis = 0, ignore_index=0)
len(gdf_3)

3550

In [29]:
gdf_3.to_file('../shp/행정경계_202402/22대총선_선거구/22대총선_선거구_동/22대총선_선거구_부천시반영후', encoding = 'utf-8')